In [5]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import joblib
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time

# Inicializar MetaTrader 5
if not mt5.initialize():
    print("Failed to initialize MT5, error code =", mt5.last_error())
    quit()

# Definir o símbolo e o intervalo de tempo
symbol = "WDOX24"
timeframe = mt5.TIMEFRAME_M1  # 1 minuto
num_bars = 500  # Número de candles para carregar

def buy_order():
    lot = 1.0
    price = mt5.symbol_info_tick(symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - 100,
        "tp": price + 100,
        "deviation": 10,
        "magic": 123456,
        "comment": "Buy order by ML model",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Compra realizada com sucesso! Preço: {price}")
    else:
        print(f"Erro ao executar compra: {result.retcode}")

def sell_order():
    lot = 1.0
    price = mt5.symbol_info_tick(symbol).bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + 100,
        "tp": price - 100,
        "deviation": 10,
        "magic": 123456,
        "comment": "Sell order by ML model",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Venda realizada com sucesso! Preço: {price}")
    else:
        print(f"Erro ao executar venda: {result.retcode}")

# Variável para armazenar a última operação
last_operation = None

# Loop principal
while True:
    try:
        # Obter dados ao vivo
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        if rates is None:
            print(f"Error: {mt5.last_error()}")
            mt5.shutdown()
            exit()
        
        # Convertendo os dados para DataFrame
        data = pd.DataFrame(rates)
        
        # Adicionando indicadores técnicos
        data['RSI'] = ta.rsi(data['close'], length=5)
        data['EMAF'] = ta.ema(data['close'], length=10)
        data['EMAM'] = ta.ema(data['close'], length=40)
        data['EMAS'] = ta.ema(data['close'], length=80)
        
        # Criando o Target (diferença entre fechamento e abertura)
        data['Target'] = data['close'] - data['open']
        data['Target'] = data['Target'].shift(-1)
        
        # Criando a TargetClass (1 para alta, 0 para queda)
        data['TargetClass'] = [1 if x > 0 else 0 for x in data['Target']]

        # Coluna 'TargetNextClose' que contém o valor de fechamento do próximo candle
        data['TargetNextClose'] = data['close'].shift(-1)
        
        # Limpeza dos dados
        data.dropna(inplace=True)
        data.reset_index(drop=True, inplace=True)
        data.drop(['time', 'spread', 'tick_volume', 'real_volume'], axis=1, inplace=True)
        
        # Normalização
        sc = MinMaxScaler(feature_range=(0, 1))
        data_scaled = sc.fit_transform(data[['open', 'high', 'low', 'close', 'RSI', 'EMAF', 'EMAM', 'EMAS']])
        
        # Preparar dados para previsão
        backcandles = 5
        X_live = []
        for j in range(data_scaled.shape[1]):
            X_live.append([])
            for i in range(backcandles, data_scaled.shape[0]):
                X_live[j].append(data_scaled[i-backcandles:i, j])
        
        X_live = np.moveaxis(X_live, [0], [2])
        X_live = np.array(X_live)
        
        # Carregar modelo e fazer previsão
        model, ref_cols, target = joblib.load("modelWDO2024_past01.pkl")
        predictions = model.predict(X_live)
        
        # Obter última previsão e dados relevantes
        current_prediction = float(predictions[-1, 0])  # Extraindo o valor da última previsão
        current_price = mt5.symbol_info_tick(symbol).last
        current_target = float(data['TargetNextClose'].iloc[-1])
        
        print(f"Previsão atual: {current_prediction:.4f}, Target Next Close atual: {current_target:.4f}, Preço atual: {current_price}")
        
        # Lógica de trading baseada no Target
        if current_target > 0:  # Target positivo indica tendência de alta
            if last_operation != "BUY":
                print(f"Sinal de COMPRA detectado - Target: {current_target:.4f}")
                buy_order()
                last_operation = "BUY"
        elif current_target < 0:  # Target negativo indica tendência de baixa
            if last_operation != "SELL":
                print(f"Sinal de VENDA detectado - Target: {current_target:.4f}")
                sell_order()
                last_operation = "SELL"
        else:
            print(f"Aguardando sinal mais claro do mercado... Target atual: {current_target:.4f}")
        
        # Aguardar antes da próxima iteração
        time.sleep(60)
        
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")
        time.sleep(60)
        continue

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Previsão atual: 0.5896, Target Next Close atual: 5675.0000, Preço atual: 5675.0
Sinal de COMPRA detectado - Target: 5675.0000
Compra realizada com sucesso! Preço: 5675.5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Previsão atual: 0.6078, Target Next Close atual: 5678.0000, Preço atual: 5677.5


KeyboardInterrupt: 